Щоб запустити наступні записники, якщо ви ще цього не зробили, вам потрібно встановити ключ openai у файлі .env як `OPENAI_API_KEY`

In [1]:
%pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:

import os
import pandas as pd
import requests
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("AIMLAPI_KEY","")
assert API_KEY, "ERROR: AIMLAPI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Далі ми завантажимо індекс вбудовувань у Pandas DataFrame. Індекс вбудовувань зберігається в JSON-файлі під назвою `embedding_index_3m.json`. Індекс вбудовувань містить вбудовування для кожного транскрипту YouTube до кінця жовтня 2023 року.

In [3]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Завантаження індексу відеосесій
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Далі ми створимо функцію під назвою `get_videos`, яка шукатиме в індексі вбудовувань за запитом. Функція поверне 5 найрелевантніших відео, які найбільш схожі на запит. Функція працює наступним чином:

1. Спочатку створюється копія індексу вбудовувань.
2. Далі обчислюється вбудовування для запиту за допомогою API вбудовування OpenAI.
3. Потім в індексі вбудовувань створюється новий стовпчик під назвою `similarity`. Стовпчик `similarity` містить косинусну подібність між вбудовуванням запиту та вбудовуванням для кожного сегмента відео.
4. Далі індекс вбудовувань фільтрується за стовпчиком `similarity`. Індекс вбудовувань фільтрується, щоб включати тільки ті відео, які мають косинусну подібність більшу або рівну 0.75.
5. Нарешті, індекс вбудовувань сортується за стовпчиком `similarity`, і повертаються 5 найкращих відео.

In [4]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # створення копії набору даних
    video_vectors = dataset.copy()

    # отримання вбудовувань для запиту за допомогою прямого виклику API
    response = requests.post(
        "https://api.aimlapi.com/v1/embeddings",
        headers={
            "Authorization": f"Bearer {API_KEY}",
            "Content-Type": "application/json"
        },
        json={
            "model": model,
            "input": query,
            "encoding_format": "float"
        }
    )
    
    data = response.json()
    query_embeddings = data["data"][0]["embedding"]
    
    # створення нового стовпчика з обчисленою подібністю для кожного рядка
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # повернення верхніх рядків
    return video_vectors.head(rows)

Ця функція дуже проста, вона просто виводить результати пошукового запиту.

In [5]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """конвертація часу з формату 00:00:00 в секунди"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nВідео, схожі на '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Резюме: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Схожість: {row['similarity']}")
        print(f"   Доповідачі: {row['speaker']}")

1. Спочатку індекс вбудовувань завантажується в DataFrame Pandas.
2. Далі користувачу пропонується ввести запит.
3. Потім викликається функція `get_videos` для пошуку в індексі вбудовувань за запитом.
4. Нарешті, викликається функція `display_results` для відображення результатів користувачу.
5. Потім користувачу пропонується ввести інший запит. Цей процес триває, доки користувач не введе `exit`.

![](../images/notebook-search.png?WT.mc_id=academic-105485-koreyst)

Вам буде запропоновано ввести запит. Введіть запит і натисніть Enter. Застосунок поверне список відео, які відповідають запиту. Застосунок також поверне посилання на місце у відео, де знаходиться відповідь на запитання.

Ось кілька запитів, які можна спробувати:

- Що таке Azure Machine Learning?
- Як працюють згорткові нейронні мережі?
- Що таке нейронна мережа?
- Чи можу я використовувати Jupyter Notebooks з Azure Machine Learning?
- Що таке ONNX?

In [6]:
pd_vectors = load_dataset(DATASET_NAME)

# отримання запиту користувача з вводу
while True:
    query = input("Введіть запит: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


Відео, схожі на 'Machine Learning':
 - You're Not Solving the Problem You Think You're Solving
   Резюме: Join Seth Juarez as he discusses ethical concerns with AI in this episode of the...
   YouTube: https://youtu.be/-tJQm4mSh1s?t=0
   Схожість: 0.7922196485431424
   Доповідачі: Seth Juarez, Josh Lovejoy, Sarah Bird
 - You're Not Solving the Problem You Think You're Solving
   Резюме: In this video, the speaker discusses the challenges of building technology that is both responsible...
   YouTube: https://youtu.be/-tJQm4mSh1s?t=187
   Схожість: 0.8169753416732455
   Доповідачі: Seth Juarez, Josh Lovejoy, Sarah Bird
 - You're Not Solving the Problem You Think You're Solving
   Резюме: The video discusses the limitations of generalizability in machine learning models and the importance of...
   YouTube: https://youtu.be/-tJQm4mSh1s?t=373
   Схожість: 0.8161087702421701
   Доповідачі: Seth Juarez, Josh Lovejoy, Sarah Bird
 - You're Not Solving the Problem You Think You're Solving
   Ре